<a href="https://colab.research.google.com/github/deny-joefakri/Unique-Face-Scrub/blob/main/VGGNet16_Unique_Faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Lib


In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
tf.random.set_seed(1234)

In [ ]:
image_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [ ]:
from google.colab import drive  
drive.mount('/content/drive')  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/Dataset/face/face_dataset/train/'

val_dir = '/content/drive/MyDrive/Colab Notebooks/Dataset/face/face_dataset/val/'

test_dir = '/content/drive/MyDrive/Colab Notebooks/Dataset/face/face_dataset/test/'

In [ ]:
batch_size = 32
input_shape = (224, 224, 3)

# Preprocess the images


# training_set = image_generator.flow_from_directory(batch_size=batch_size,
#                                                    directory=train_dir,
#                                                    shuffle=True,
#                                                    target_size=input_shape[:2],
#                                                    class_mode="categorical")

# validation_set = image_generator.flow_from_directory(batch_size=batch_size,
#                                                    directory=val_dir,
#                                                    shuffle=True,
#                                                    target_size=input_shape[:2],
#                                                    class_mode="categorical")

# testing_set = image_generator.flow_from_directory(batch_size=batch_size,
#                                                    directory=test_dir,
#                                                    shuffle=False,
#                                                    target_size=input_shape[:2],
#                                                    class_mode="categorical")

IMG_SIZE = (224, 224)
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2,
                                   zoom_range=0.25, 
                                   horizontal_flip=True, 
                                   samplewise_center=True, 
                                   samplewise_std_normalization=True,
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(train_dir, target_size=input_shape[:2], 
                                         color_mode="rgb",
                                         batch_size=200, 
                                         shuffle=True,
                                         class_mode="categorical")

testing_set = test_datagen.flow_from_directory(test_dir, target_size=input_shape[:2], 
                                         color_mode="rgb",
                                         batch_size=64, 
                                         shuffle=True,
                                         class_mode="categorical")

validation_set = train_datagen.flow_from_directory(val_dir, target_size=input_shape[:2], 
                                         color_mode="rgb",
                                         batch_size=64, 
                                         shuffle=True,
                                         class_mode="categorical")


Found 3523 images belonging to 50 classes.
Found 250 images belonging to 50 classes.
Found 250 images belonging to 50 classes.


In [ ]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers.legacy import Adam
from keras.optimizers import SGD

BATCH_SIZE = 32
input_shape = (224, 224, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes=50

n_steps = training_set.samples // BATCH_SIZE
n_val_steps = validation_set.samples // BATCH_SIZE
n_epochs = 10

# First we'll train the model without Fine-tuning
model = create_model(input_shape, n_classes, optim_1, fine_tune=0)



# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='tl_model_v1.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

history = model.fit(training_set,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validation_set,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            verbose=1)



Epoch 1/10


InvalidArgumentError: ignored

In [ ]:
training_set.samples

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10, 5))
plt.grid(True)
plt.show()

In [ ]:
model.evaluate(testing_set)

In [ ]:
test_pred = np.argmax(model.predict(testing_set), axis=1)

In [ ]:
test_loss, test_acc = model.evaluate(testing_set, steps=len(testing_set), verbose=1)
print('Loss: %.3f' % (test_loss * 100.0))
print('Accuracy: %.3f' % (test_acc * 100.0))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

y_val = testing_set.classes
y_pred = np.argmax(model.predict(testing_set),axis=1)
print(classification_report(y_val,y_pred))

cm = confusion_matrix(y_val, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

# print(classification_report(testing_set,test_pred))
# print(confusion_matrix(testY, predict))



# print(classification_report(testing_set,test_pred))